In [4]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from joblib import dump, load
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoModel
from datasets import load_dataset
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from joblib import dump
import numpy as np

# Load dataset

In [17]:
from datasets import load_dataset

# Load a custom CSV file
data_files = {"train": "../../data/train_data.csv", "test": "../../data/test_data.csv"}
dataset = load_dataset("csv", data_files=data_files)

# Inspect the first few samples
print(dataset["train"][0])
# Example: Mapping string labels to integers
label_mapping = {
    'notcb': 0,
    'gender': 1,
    'ethnicity': 2,
    'religion': 3,
    'age' : 4,
    'other': 5
}
dataset = dataset.map(lambda x: {"label": label_mapping[x["label"]]})

# Verify the mapping
print(dataset)

Generating train split: 38408 examples [00:00, 463331.57 examples/s]
Generating test split: 9602 examples [00:00, 433325.52 examples/s]


{'text': 'heyy [USER] &amp; [USER] , i just saw the truth about / E1 of adelaines mental health series. it was so moving and relatable for me. i had trouble my last year of high school with this one girl. she bullied me because of my disability.', 'label': 'age'}


Map: 100%|██████████| 9602/9602 [00:00<00:00, 35906.51 examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 38408
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 9602
    })
})


In [18]:
model_name = "distilroberta-base"

# tokenize

In [20]:
dataset["test"]

Dataset({
    features: ['text', 'label'],
    num_rows: 9602
})

In [16]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples): #padding false becase of data collator later
    for text in examples["text"]:
        if text is not None and text != "":
            continue
        else:
            print(text)
            return
    return tokenizer(examples["text"], padding=False, truncation=True, max_length=200)

# Apply the tokenizer to the dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Inspect tokenized samples
print(tokenized_datasets["train"][0])

Map:  39%|███▉      | 15000/38408 [00:00<00:00, 57123.38 examples/s]

None
None
None
None


Map:  78%|███████▊  | 30000/38408 [00:00<00:00, 49060.43 examples/s]

None
None
None
None


Map: 100%|██████████| 38408/38408 [00:00<00:00, 49222.33 examples/s]


None


Map: 100%|██████████| 9602/9602 [00:00<00:00, 40667.16 examples/s]

None
{'text': 'heyy [USER] &amp; [USER] , i just saw the truth about / E1 of adelaines mental health series. it was so moving and relatable for me. i had trouble my last year of high school with this one girl. she bullied me because of my disability.', 'label': 4, 'input_ids': [0, 12229, 219, 646, 47955, 742, 359, 3914, 131, 646, 47955, 742, 2156, 939, 95, 794, 5, 3157, 59, 1589, 381, 134, 9, 2329, 523, 1851, 293, 2536, 474, 651, 4, 24, 21, 98, 1375, 8, 6258, 18564, 13, 162, 4, 939, 56, 3605, 127, 94, 76, 9, 239, 334, 19, 42, 65, 1816, 4, 79, 21745, 162, 142, 9, 127, 11096, 4, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [3]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=6)

print(model.config)

C:\Users\SEBASTIAN\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\SEBASTIAN\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling 

KeyboardInterrupt: 

KeyboardInterrupt: 

# How to Custom Model

In [ ]:
# import torch.nn as nn
#
# class CustomBERTModel(nn.Module):
#     def __init__(self, pretrained_model_name, num_labels):
#         super(CustomBERTModel, self).__init__()
#         self.bert = AutoModel.from_pretrained(pretrained_model_name)
#         self.dropout = nn.Dropout(0.3)
#         self.fc = nn.Linear(self.bert.config.hidden_size, num_labels)
#
#     def forward(self, input_ids, attention_mask):
#         output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
#         pooled_output = self.dropout(output[1])  # Applying dropout
#         logits = self.fc(pooled_output)  # Adding a fully connected layer
#         return logits
#
# # Initialize the custom model
# custom_model = CustomBERTModel("bert-base-uncased", num_labels=4)

# Train arguments

In [ ]:
from transformers import TrainingArguments

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",           # Directory for saving model checkpoints
    eval_strategy="epoch",     # Evaluate at the end of each epoch
    save_strategy="epoch",
    learning_rate=5e-5,              # Start with a small learning rate
    per_device_train_batch_size=16,  # Batch size per GPU
    per_device_eval_batch_size=16,
    num_train_epochs=3,              # Number of epochs
    weight_decay=0.01,               # Regularization
    save_total_limit=2,              # Limit checkpoints to save space
    load_best_model_at_end=True,     # Automatically load the best checkpoint
    logging_dir="./logs",            # Directory for logs
    logging_steps=100,               # Log every 100 steps
    fp16=True                        # Enable mixed precision for faster training
)

print(training_args)

In [ ]:
from transformers import Trainer
from evaluate import load
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np

# Load a metric (recall and precision for our dataset ania)
metric = load("recall")

# Define a custom compute_metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # Calculate accuracy
    accuracy = accuracy_score(labels, predictions)

    # Calculate precision, recall, f1 with macro averaging (treats all classes equally)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average='macro', zero_division=0
    )

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
trainer = Trainer(
    model=model,                        # Pre-trained BERT model
    args=training_args,                 # Training arguments
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,        # Efficient batching
    compute_metrics=compute_metrics,    # Custom metric
)
from transformers import EarlyStoppingCallback

trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=2))


In [ ]:
trainer.train()

In [ ]:
results = trainer.evaluate()
print(results)

In [ ]:
from matplotlib import pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

# Generate predictions
predictions = trainer.predict(tokenized_datasets["test"])
predicted_labels = predictions.predictions.argmax(axis=-1)

# Classification report
print(classification_report(tokenized_datasets["test"]["label"], predicted_labels))

# Confusion matrix
cm = confusion_matrix(tokenized_datasets["test"]["label"], predicted_labels)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["NO CB", "sexual","race", "religion"])
disp.plot(cmap="Blues")  # Optional: set a color map
plt.tight_layout()
plt.savefig("confusion_matrix.png", dpi=300)  # You can change the name or dpi as needed
plt.close()  # Close the plot to free memory if you're in a loop

In [ ]:
# Inspect misclassified samples
for idx, (pred, label) in enumerate(zip(predicted_labels, tokenized_datasets["test"]["label"])):
    if pred != label:
        print(f"Index: {idx}, Predicted: {pred}, Actual: {label}")
        print(tokenized_datasets["test"][idx]["text"])

In [ ]:
import os

# Add this after trainer.train() and evaluation
# Save the trained model and tokenizer
os.makedirs("./bert_model", exist_ok=True)
trainer.save_model("../../models/distil_roberta")
tokenizer.save_pretrain("../../models/distil_roberta")
print("Model saved successfully!")